## Making Wiki tool



In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import Tool
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os


## Making wikipedia tool



In [24]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wikipedia=WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia


WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/opt/anaconda3/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250))

## Loading text and saving in text documents  from the link 

In [25]:
loader=WebBaseLoader(web_paths=("https://www.ibm.com/topics/artificial-intelligence",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                      class_=("rich-text text")    
                     )))
text_documents2=loader.load()
text_documents2[0].page_content.splitlines()[:5]

['',
 'Artificial intelligence (AI) is technology that enables computers and machines to simulate human learning, comprehension, problem solving, decision making, creativity and autonomy.',
 '',
 'Applications and devices equipped with AI can see and identify objects. They can understand and respond to human language. They can learn from new information and experience. They can make detailed recommendations to users and experts.\xa0They can act independently, replacing the need for human intelligence or intervention (a classic example being a self-driving car).',
 'But in 2024, most AI researchers, practitioners and most AI-related headlines are focused on breakthroughs in generative AI\xa0(gen AI), a technology that can create original text, images, video and other content. To fully understand generative AI, it’s important to first understand the technologies on which generative AI tools are built: machine learning\xa0(ML) and deep learning.']

## Splitting the text documents in small parts

In [26]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(text_documents2)
documents[:1]

[Document(metadata={'source': 'https://www.ibm.com/topics/artificial-intelligence'}, page_content='Artificial intelligence (AI) is technology that enables computers and machines to simulate human learning, comprehension, problem solving, decision making, creativity and autonomy.\n\nApplications and devices equipped with AI can see and identify objects. They can understand and respond to human language. They can learn from new information and experience. They can make detailed recommendations to users and experts.\xa0They can act independently, replacing the need for human intelligence or intervention (a classic example being a self-driving car).\nBut in 2024, most AI researchers, practitioners and most AI-related headlines are focused on breakthroughs in generative AI\xa0(gen AI), a technology that can create original text, images, video and other content. To fully understand generative AI, it’s important to first understand the technologies on which generative AI tools are built: mach

## Embedding the small ext documents and saving them in vector store 

In [27]:
database=FAISS.from_documents(documents,HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
retriever=database.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x34468b770>, search_kwargs={})

## Making Arxiv Tool 

In [29]:
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=150)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

## Making calculator tool

In [30]:
def basic_calculator(expression: str) -> str:
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"
calculator_tool = Tool(
    name="Calculator",
    func=basic_calculator,
    description="Evaluate a basic math expression. Example: '12 / (2 + 1)'"
)

## Creating web search tool

In [31]:
search_tool = DuckDuckGoSearchRun()


web_search = Tool(
    name="web_search",
    description="Use this tool to search the internet and return text summaries from DuckDuckGo.",
    func=lambda input: search_tool.run(input)

)

## Combining tools

In [32]:
tools=[wikipedia,AI_search,arxiv,calculator_tool]

## Using GROQ llm 

In [33]:
load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="qwen-qwq-32b") 

## Creating prompt

In [34]:
prompt=ChatPromptTemplate.from_template(""" You are an intelligent AI assistant. You have access to the following tools:

- Use them if they are helpful.
- Prefer using them when answering technical or factual questions.
- Do not guess answers if the tools can help.
<context>
{context}
</context>
Question:{input}
{agent_scratchpad}                                 
                                           """

 )

## Agents bucket to give name and description to llm 

In [35]:
agent=create_openai_tools_agent(llm,tools,prompt)

## Execute the llm output and give back the result in prompt  

In [36]:
agent_excutor=AgentExecutor(agent=agent,tools=tools,verbose=True,)
repr(agent_excutor).splitlines()[:5]


['AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={',
 "  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))",
 '})',
 "| ChatPromptTemplate(input_variables=['agent_scratchpad', 'context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'context', 'input'], input_types={}, partial_variables={}, template=' You are an intelligent AI assistant. You have access to the following tools:\\n\\n- Use them if they are helpful.\\n- Prefer using them when answering technical or factual questions.\\n- Do not guess answers if the tools can help.\\n<context>\\n{context}\\n</context>\\nQuestion:{input}\\n{agent_scratchpad}                                 \\n                                           '), additional_kwargs={})])",
 '| RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions obj

## Finally invoking the agent Executor

In [ ]:
result=agent_excutor.invoke({"input":"what is artificial intelligence","context": "" })
result["output"] 



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'artificial intelligence'}`


Page: Artificial intelligence
Summary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-makinArtificial Intelligence (AI) refers to the development of computer systems capable of performing tasks that typically require human intelligence. These tasks include learning from data, reasoning, problem-solving, perception (e.g., recognizing images or speech), and making decisions. AI systems achieve this through algorithms, machine learning models, and large datasets, enabling them to improve over time without explicit programming for every scenario.

Common applications of AI include:
- **Virtual assistants** (e.g., Siri, Alexa)
- **Autonomous vehicles**
- **Recommendation systems** (e.g., Netflix, Spotify)
- **Medical diagnostic

'Artificial Intelligence (AI) refers to the development of computer systems capable of performing tasks that typically require human intelligence. These tasks include learning from data, reasoning, problem-solving, perception (e.g., recognizing images or speech), and making decisions. AI systems achieve this through algorithms, machine learning models, and large datasets, enabling them to improve over time without explicit programming for every scenario.\n\nCommon applications of AI include:\n- **Virtual assistants** (e.g., Siri, Alexa)\n- **Autonomous vehicles**\n- **Recommendation systems** (e.g., Netflix, Spotify)\n- **Medical diagnostics**\n- **Natural language processing** (e.g., chatbots, translation tools)\n\nThe field of AI aims to create systems that can mimic or surpass human-like cognitive abilities, though most current AI is *narrow* (task-specific), like voice recognition, rather than *general* (human-like consciousness or adaptability across tasks).'